### Serial to Parallel: OpenMP

The simplest and most common parallel pattern is to take a serial program and convert it into a parallel program.

* My code’s not running fast enough:
  * Video: data delays produce jitter, stalls
  * Web: page render time causes user loss, discomfort
  * Batch processing, indexing, analysis not completing in time 
  * High-throughput finance: other models running faster and beating mine to a decision -> lose arbitrage opportunity
* This leads to a natural software engineering process
  * Profile code: find out what’s slow
  * Parallelize slow part(s) only
  * Migrate from serial implementation to parallel implementation
* It's  not the best process
  * Serial to parallel doesn’t produce the best designs
  * Best parallel implementation may require a totally different design with no incremental refatoring from serial implementat
* Just the easiest
  * Compared to a clean-slate redesign


